In [ ]:
# Given a certain number of blocks in order to cover a set (represented for example as a line), we want to pick the minimum number of blocks (or smaller lines).
# the problem is discretized, so instead of having a continuos line we consider blocks of continuous data
# we may use a set with all taken pieces {T} and another one with not taken ones {N} (and can be seen as the state of the problem)

In [ ]:
""" The code addresses a problem where there are multiple sets (SETS) of boolean values (True or False) representing elements. The goal is to find a combination of these sets such that, when combined using a logical OR operation, all elements are True.

It generates 10 (NUM_SETS) sets, each containing 5 (PROBLEM_SIZE) random boolean values, where the chance of a value being True is 0.3.
It then uses a depth-first search (DFS) approach (indicated by using a LifoQueue) to explore the combinations of these sets.
The goal_check function verifies whether a given combination of sets satisfies the objective.
The solution state contains two fields:
taken: Sets that are selected in the combination.
not_taken: Sets that are not selected in the combination.
The DFS continues until a solution (combination of sets) is found or all possibilities are exhausted. """

In [ ]:
from random import random
from queue import PriorityQueue, SimpleQueue, LifoQueue
from functools import reduce
import numpy as np
from collections import namedtuple,defaultdict

In [ ]:
# Define the problem size and the number of sets
PROBLEM_SIZE = 5 # can be defined as constant (but in reality can always be modified, it's just syntax)
NUMBER_SETS = 10

# Generate NUM_SETS number of sets with PROBLEM_SIZE boolean values each
# Each value has a 0.3 probability of being True
SETS = tuple(np.array([random() < .3 for _ in range(PROBLEM_SIZE)]) for _ in range(NUMBER_SETS)) # .2 is chance of an element to be true

# Define a named tuple to represent the state with fields 'taken' and 'not_taken'
State = namedtuple('State', ['taken', 'not_taken'])

In [ ]:
# Function to check if a given state meets the goal (all elements combined are True)
def goal_check(state):
    return np.all(reduce(np.logical_or,[SETS[i] for i in state.taken], np.array([False for _ in range(PROBLEM_SIZE)]))) 

In [ ]:
# Check if the problem is solvable
assert goal_check(State(set(range(NUMBER_SETS)), set())), "Problem not solvable"

In [ ]:
# state = ({1,3,5}, {0,2,4,6,7}) # (Taken, not taken)

In [ ]:
# [SETS[i] for i in state[0]]

In [ ]:
# sum(SETS[i] for i in state[0])

In [ ]:
# np.all(reduce(np.logical_or,[SETS[i] for i in state[0]])) # final assessment that all spaces in tuple have been covered

In [ ]:
#state = (set(range(NUMBER_SETS)), set())
#state

In [ ]:
#goal_check(state)

In [ ]:
#sum([SETS[i] for i in state[0]]) # overlapping of sets seen

In [ ]:
# Use a LifoQueue for depth-first search
frontier = LifoQueue() # Simple -> breath first approach, with priority queue instead random elements from the frontier get extracted, Lifo in this case is better
frontier.put(State(set(), set(range(NUMBER_SETS))))
frontier.get()

In [ ]:
# Counter to keep track of steps taken
counter = 0

# Start DFS until a goal state is reached or all possibilities are explored
current_state = frontier.get()
while not goal_check(current_state):
    counter += 1
    for action in current_state[1]:
        new_state = State(current_state.taken ^ {action}, current_state.not_taken ^ {action})
        frontier.put(new_state)
    current_state = frontier.get()

# Print the solution state and verify it meets the goal
print(f"Solved in {counter:,} steps")
# still won't work from time to time, but same on prof code

In [ ]:
goal_check(current_state)

In [ ]:
# try then implementing Dijkstra's algorithms

# GPT generated

def dijkstra(start):
    # Dictionary to store the shortest distances from the start state
    shortest_distance = defaultdict(lambda: float('inf')) # that is infinite, so if nothing found instead of error I return infinite
    shortest_distance[start] = 0

    # Priority queue to manage states based on distance
    pq = PriorityQueue()
    pq.put((0, start))

    while not pq.empty():
        current_distance, current_state = pq.get()

        if current_distance > shortest_distance[current_state]:
            continue

        for action in current_state.not_taken:
            new_state = State(current_state.taken | frozenset({action}), current_state.not_taken - frozenset({action}))

            distance = current_distance + 1  # using a mock weight of 1 for each transition
            if distance < shortest_distance[new_state]:
                shortest_distance[new_state] = distance
                pq.put((distance, new_state))

    return shortest_distance

# Get shortest distances from the start state to all other states
distances = dijkstra(State(frozenset(), frozenset(range(NUMBER_SETS))))

# Find a state that meets the goal and has the shortest distance
goal_state = min((state for state in distances if goal_check(state)), key=lambda s: distances[s])

print(f"Solved with a distance of {distances[goal_state]}")
print(goal_state)
print(goal_check(goal_state))